# ECE analysis

In [1]:
import utils.utils as utils

from utils.constants import CIFAR10_DICT_MODELS_TO_ID, IMAGENET_DICT_MODELS_TO_ID

import numpy as np
import plot_results as ps

In [2]:
robustness_level = "robust"
ds_name = "imagenet"

ece_save_path = "figures/calibration_hist/"
ece_dict_name = f"calibration_curve_{robustness_level}_{ds_name}"
ece_dict = utils.my_load(f"{ece_save_path}{ece_dict_name}.pkl")

In [3]:
entropy_save_path = "figures/violin_plots/"
entropy_dict_name = f"violinplots_{robustness_level}_{ds_name}"
entropy_dict = utils.my_load(f"{entropy_save_path}{entropy_dict_name}.pkl")

In [4]:
dict_to_model = CIFAR10_DICT_MODELS_TO_ID if ds_name == "cifar10" else IMAGENET_DICT_MODELS_TO_ID

In [5]:
dict_to_model

{'engstrom2019imgnet': {'paper_id': '\\engstromimagenetID',
  'rb_rank': '6',
  'clean_acc': '62.56\\%',
  'rob_acc': '29.22\\%'},
 'salman2020R18': {'paper_id': '\\salmanID',
  'rb_rank': '8',
  'clean_acc': '52.92\\%',
  'rob_acc': '25.32\\%'},
 'salman2020R50': {'paper_id': '\\salmanIDdue',
  'rb_rank': '5',
  'clean_acc': '64.02\\%',
  'rob_acc': '34.96\\%'},
 'wong2020': {'paper_id': '\\wongfastID',
  'rb_rank': '7',
  'clean_acc': '55.62\\%',
  'rob_acc': '26.24\\%'},
 'Liu2023swinB': {'paper_id': '\\liuswinb',
  'rb_rank': '3',
  'clean_acc': '76.16\\%',
  'rob_acc': '56.16\\%'},
 'Liu2023swinL': {'paper_id': '\\liuswinl',
  'rb_rank': '1',
  'clean_acc': '78.92\\%',
  'rob_acc': '59.56\\%'},
 'Liu2023convNextB': {'paper_id': '\\liuconvnb',
  'rb_rank': '4',
  'clean_acc': '76.02\\%',
  'rob_acc': '55.82\\%'},
 'Liu2023convNextL': {'paper_id': '\\liuconvnl',
  'rb_rank': '2',
  'clean_acc': '78.02\\%',
  'rob_acc': '58.48\\%'}}

In [6]:
# print("ECE")
# for idx, val in ece_dict.items():
#     print(CIFAR10_DICT_MODELS_TO_ID[idx], end=" ")
#     for idx, val in val.items():
#         print(f"& {val:.3f}", end=" ")
#         
#     print("\\\\")

In [7]:
atk_type = "under_ece"
attack_ece = {k: v[atk_type] for k, v in ece_dict.items()}
sort_attack = dict(sorted(attack_ece.items(), key=lambda item: item[1]))

# print(atk_type)
# counter = 0
# for idx, item in sort_attack.items():
#     print(f"{idx} --> {item}")
#     counter += item
#     
# print(counter/len(sort_attack))

In [8]:
atk_type = "over_ece"
attack_ece = {k: v[atk_type] for k, v in ece_dict.items()}
sort_attack = dict(sorted(attack_ece.items(), key=lambda item: item[1]))

# print(atk_type)
# counter = 0
# for idx, item in sort_attack.items():
#     print(f"{idx} --> {item}")
#     counter += item
#     
# print(counter/len(sort_attack))

In [9]:
# ECE MAX DIFFERENCE Uatk and Oatk
difference_ece_dict = {k: abs(v["under_ece"]-v["clean_ece"]) + abs(v["clean_ece"]-v["over_ece"]) for k, v in ece_dict.items()}
sort_difference = dict(sorted(difference_ece_dict.items(), key=lambda item: item[1]))

# print("ECE delta")
# 
# for idx, item in sort_difference.items():
#     print(f"{idx} --> {item}")

### ECE table all metrics (Clean - Over - Umder)

Per formattare bene i numeri
$\displaystyle 0.222\genfrac{}{}{-.4pt}{}{+0.052}{-0.095}$

In [10]:
clean_ece = {k: v["clean_ece"] for k, v in ece_dict.items()}
under_ece = {k: v["under_ece"] for k, v in ece_dict.items()}
over_ece = {k: v["over_ece"] for k, v in ece_dict.items()}

In [11]:
clean_entropy = {k: v["clean"] for k, v in entropy_dict.items()}
under_entropy = {k: v["uatk"] for k, v in entropy_dict.items()}
over_entropy = {k: v["oatk"] for k, v in entropy_dict.items()}

In [12]:
clean_area = {k: v["clean_area"] for k, v in ece_dict.items()}
under_area = {k: v["under_area"] for k, v in ece_dict.items()}
over_area = {k: v["over_area"] for k, v in ece_dict.items()}

In [13]:
# Displacement
# for idx, item in clean_ece.items():
#     print(f"{CIFAR10_DICT_MODELS_TO_ID[idx]} --> clean: {item:.3f} under: +{(under_ece[idx]-item):.3f} over: -{(item - over_ece[idx]):.3f}")

In [14]:
# Upper - lower - Delta
# for idx, item in clean_ece.items():
#     print(f"{CIFAR10_DICT_MODELS_TO_ID[idx]} --> clean: {item:.3f} under: {(under_ece[idx]):.3f} over: {(over_ece[idx]):.3f} diff: {(under_ece[idx] - over_ece[idx]):.3f}")

## Generate Latex table

CONTROLLA SE STAI FACENDO CON ECE O NUOVA METRICA

In [38]:
model_names = [dict_to_model[mod_name]["paper_id"] for mod_name in dict_to_model.keys()]
model_names

['\\engstromimagenetID',
 '\\salmanID',
 '\\salmanIDdue',
 '\\wongfastID',
 '\\liuswinb',
 '\\liuswinl',
 '\\liuconvnb',
 '\\liuconvnl']

In [39]:
model_x_row = 8
total_model = len(clean_ece.items())

tot_rows = total_model//model_x_row + 1 if total_model%model_x_row != 0 else 1

utk_ece_diff_list=[]
otk_ece_diff_list=[]

utk_entroypy_diff_list=[]
otk_entroypy_diff_list=[]

model_names = [dict_to_model[mod_name]["paper_id"] for mod_name in dict_to_model.keys()]


model_ids = list(dict_to_model.keys())
model_ids = [f"C{i+1}" if ds_name == "cifar10" else f"I{i+1}" for i in range(len(dict_to_model.keys()))]


print("\\begin{table}[]")
for row in range(tot_rows):    
    # intestazione
    print("\\resizebox{\\textwidth}{!}{\n\\begin{tabular}{lllllllllll}")
    print("&  ", end=" ")
    
    for i in range(model_x_row):
        idx = i+(row*model_x_row)
        if idx >= total_model:
            break
        print(f"& \multicolumn{{1}}{{c}}{{{model_names[idx]}}}", end=" ")
    print(f"\\\\ \cline{{3-{model_x_row+2}}}")
    
    
    # --------------------------------- FAI ATTENZIONE QUA ------------------------------------
    
    # clean_ece_values = list(clean_ece.values())
    # under_ece_values = list(under_ece.values())
    # over_ece_values = list(over_ece.values())

    clean_ece_values = [-x for x in list(clean_area.values())]
    under_ece_values = [-x for x in list(under_area.values())]
    over_ece_values = [-x for x in list(over_area.values())]

    # --------------------------------- FAI ATTENZIONE QUA ------------------------------------


    # UNDER ATK
    print("\multicolumn{1}{c}{\multirow{3}{*}{CE}}  & Utk  ", end= " ") # <----- changed from ECE
    for i in range(model_x_row):
        idx = i+(row*model_x_row)
        if idx >= total_model:
            break
        # print(f"& {under_ece_values[idx]:.3f}", end=" ")
        cell_value = round(utk_diff:=(under_ece_values[idx] - clean_ece_values[idx]), 3)
        # print(f"---{utk_diff}---")
        utk_ece_diff_list.append(utk_diff)
        print("& " + ("$\\uparrow +" if cell_value >=0 else "$\downarrow ") +str(cell_value) + "$", end=" ")
    print("\\\\")
    
    # CLEAN
    print("\multicolumn{1}{c}{} & Clean  ", end= " ")
    for i in range(model_x_row):
        idx = i+(row*model_x_row)
        if idx >= total_model:
            break
        print(f"& ${clean_ece_values[idx]:.3f}$", end=" ")
    print("\\\\")
    
    
    # OVER
    print("\multicolumn{1}{c}{} & Oatk  ", end= " ")
    for i in range(model_x_row):
        idx = i+(row*model_x_row)
        if idx >= total_model:
            break
        #print(f"& {over_ece_values[idx]:.3f}", end=" ")
        cell_value = round(otk_diff:=(over_ece_values[idx] - clean_ece_values[idx]), 3)
        otk_ece_diff_list.append(otk_diff)
        print("& " + ("$\\uparrow +" if cell_value >=0 else "$\downarrow ") + str(cell_value) +"$", end=" ")
    
    print(f"\\\\ \cdashline{{1-{model_x_row+2}}}")
    
    
    # ENTROPYYYYYYYYYYYYYYYYYYYYYYY
    under_entropy_values = list(under_entropy.values())
    clean_entropy_values = list(clean_entropy.values())
    over_entropy_values = list(over_entropy.values())

    # UNDER ATK
    print("\multicolumn{1}{c}{\multirow{3}{*}{H(x)}}  & Utk  ", end= " ")
    for i in range(model_x_row):
        idx = i+(row*model_x_row)
        if idx >= total_model:
            break
        #print(f"& ${under_entropy_values[idx]:.3f}$", end=" ")
        cell_value = round(utk_ent:=(under_entropy_values[idx] - clean_entropy_values[idx]), 3)
        utk_entroypy_diff_list.append(utk_ent)
        print("& " + ("$\\uparrow +" if cell_value >=0 else "$\downarrow ") + str(cell_value) +"$", end=" ")
    print("\\\\")
    
    # CLEAN
    print("\multicolumn{1}{c}{} & Clean  ", end= " ")
    for i in range(model_x_row):
        idx = i+(row*model_x_row)
        if idx >= total_model:
            break
        print(f"& ${clean_entropy_values[idx]:.3f}$", end=" ")
    print("\\\\")
    
    
    # OVER
    print("\multicolumn{1}{c}{} & Oatk  ", end= " ")
    for i in range(model_x_row):
        idx = i+(row*model_x_row)
        if idx >= total_model:
            break
        #print(f"& ${over_entropy_values[idx]:.3f}$", end=" ")
        cell_value = round(otk_ent:=(over_entropy_values[idx] - clean_entropy_values[idx]), 3)
        otk_entroypy_diff_list.append(otk_ent)
        print("& " + ("$\\uparrow +" if cell_value >=0 else "$\downarrow ") + str(cell_value) +"$", end=" ")
    
    #print(f"\\\\ \cline{{1-{model_x_row+2}}}")
    print("\n\end{tabular}}\n")
    
print("\caption{INSERT CAPTION}\n"+f"\label{{{'tab:cifar' if ds_name=='cifar10' else 'tab:imagenet'}}}\n"+"\end{table}")
    
    
print("\n")

\begin{table}[]
\resizebox{\textwidth}{!}{
\begin{tabular}{lllllllllll}
&   & \multicolumn{1}{c}{\engstromimagenetID} & \multicolumn{1}{c}{\salmanID} & \multicolumn{1}{c}{\salmanIDdue} & \multicolumn{1}{c}{\wongfastID} & \multicolumn{1}{c}{\liuswinb} & \multicolumn{1}{c}{\liuswinl} & \multicolumn{1}{c}{\liuconvnb} & \multicolumn{1}{c}{\liuconvnl} \\ \cline{3-10}
\multicolumn{1}{c}{\multirow{3}{*}{CE}}  & Utk   & $\uparrow +0.137$ & $\uparrow +0.093$ & $\uparrow +0.12$ & $\uparrow +0.121$ & $\uparrow +0.127$ & $\uparrow +0.106$ & $\uparrow +0.1$ & $\uparrow +0.084$ \\
\multicolumn{1}{c}{} & Clean   & $0.058$ & $0.107$ & $0.076$ & $-0.009$ & $0.187$ & $0.163$ & $0.167$ & $0.157$ \\
\multicolumn{1}{c}{} & Oatk   & $\downarrow -0.261$ & $\downarrow -0.224$ & $\downarrow -0.226$ & $\downarrow -0.274$ & $\downarrow -0.175$ & $\downarrow -0.094$ & $\downarrow -0.156$ & $\downarrow -0.135$ \\ \cdashline{1-10}
\multicolumn{1}{c}{\multirow{3}{*}{H(x)}}  & Utk   & $\uparrow +1.196$ & $\uparrow +0

# Metric considerations: GUARDA SE AREA (SECE) O ECE

Ora stiamo lavorando con SECE

## Mean values

In [16]:
print("underconfidence:", np.mean(under_ece_values), np.std(under_ece_values))
print("clean:", np.mean(clean_ece_values), np.std(clean_ece_values))
print("overconfidence:", np.mean(over_ece_values), np.std(over_ece_values))


underconfidence: 0.2244405641981284 0.05824323507565784
clean: 0.11326623724395177 0.06366380936954255
overconfidence: -0.07990010932902804 0.11845842135245513


In [17]:
# DIFF
print("mean value diff")
print("under-clean: ",np.mean(under_ece_values)-np.mean(clean_ece_values))
print("over-clean: ", np.mean(over_ece_values)-np.mean(clean_ece_values))

mean value diff
under-clean:  0.11117432695417664
over-clean:  -0.19316634657297982


In [18]:
# for i in (np.array(under_ece_values)).argsort():
#     print(model_names[i], model_ids[i], clean_ece_values[i])

## Models vs UNDERconfidence

In [19]:
under_clean_diff = np.array(under_ece_values)-np.array(clean_ece_values)

for i in under_clean_diff.argsort():
    print(model_names[i], model_ids[i], under_clean_diff[i])

{'paper_id': '\\liuconvnl', 'rb_rank': '2', 'clean_acc': '78.02\\%', 'rob_acc': '58.48\\%'} I8 0.08429830500343816
{'paper_id': '\\salmanID', 'rb_rank': '8', 'clean_acc': '52.92\\%', 'rob_acc': '25.32\\%'} I2 0.09293979359511283
{'paper_id': '\\liuconvnb', 'rb_rank': '4', 'clean_acc': '76.02\\%', 'rob_acc': '55.82\\%'} I7 0.10040161917731164
{'paper_id': '\\liuswinl', 'rb_rank': '1', 'clean_acc': '78.92\\%', 'rob_acc': '59.56\\%'} I6 0.10637564423726872
{'paper_id': '\\salmanIDdue', 'rb_rank': '5', 'clean_acc': '64.02\\%', 'rob_acc': '34.96\\%'} I3 0.11997267980221661
{'paper_id': '\\wongfastID', 'rb_rank': '7', 'clean_acc': '55.62\\%', 'rob_acc': '26.24\\%'} I4 0.12148994592670352
{'paper_id': '\\liuswinb', 'rb_rank': '3', 'clean_acc': '76.16\\%', 'rob_acc': '56.16\\%'} I5 0.12676749706082044
{'paper_id': '\\engstromimagenetID', 'rb_rank': '6', 'clean_acc': '62.56\\%', 'rob_acc': '29.22\\%'} I1 0.13714913083054128


## Models vs OVERconfidence

In [20]:
over_clean_diff = np.array(over_ece_values)-np.array(clean_ece_values)

for i in over_clean_diff.argsort()[::-1]:
    print(model_names[i], model_ids[i], over_clean_diff[i])

{'paper_id': '\\liuswinl', 'rb_rank': '1', 'clean_acc': '78.92\\%', 'rob_acc': '59.56\\%'} I6 -0.09365021394519134
{'paper_id': '\\liuconvnl', 'rb_rank': '2', 'clean_acc': '78.02\\%', 'rob_acc': '58.48\\%'} I8 -0.1354339391589165
{'paper_id': '\\liuconvnb', 'rb_rank': '4', 'clean_acc': '76.02\\%', 'rob_acc': '55.82\\%'} I7 -0.15641427890118212
{'paper_id': '\\liuswinb', 'rb_rank': '3', 'clean_acc': '76.16\\%', 'rob_acc': '56.16\\%'} I5 -0.17526793891377745
{'paper_id': '\\salmanID', 'rb_rank': '8', 'clean_acc': '52.92\\%', 'rob_acc': '25.32\\%'} I2 -0.22371340323984623
{'paper_id': '\\salmanIDdue', 'rb_rank': '5', 'clean_acc': '64.02\\%', 'rob_acc': '34.96\\%'} I3 -0.22620382634690034
{'paper_id': '\\engstromimagenetID', 'rb_rank': '6', 'clean_acc': '62.56\\%', 'rob_acc': '29.22\\%'} I1 -0.26088675627065816
{'paper_id': '\\wongfastID', 'rb_rank': '7', 'clean_acc': '55.62\\%', 'rob_acc': '26.24\\%'} I4 -0.2737604158073664


## OVER vs UNDER

In [21]:
under_over_diff = np.array(under_ece_values)-np.array(over_ece_values)

for i in under_over_diff.argsort():
    print(model_names[i], model_ids[i], under_over_diff[i])

{'paper_id': '\\liuswinl', 'rb_rank': '1', 'clean_acc': '78.92\\%', 'rob_acc': '59.56\\%'} I6 0.20002585818246005
{'paper_id': '\\liuconvnl', 'rb_rank': '2', 'clean_acc': '78.02\\%', 'rob_acc': '58.48\\%'} I8 0.21973224416235465
{'paper_id': '\\liuconvnb', 'rb_rank': '4', 'clean_acc': '76.02\\%', 'rob_acc': '55.82\\%'} I7 0.25681589807849375
{'paper_id': '\\liuswinb', 'rb_rank': '3', 'clean_acc': '76.16\\%', 'rob_acc': '56.16\\%'} I5 0.3020354359745979
{'paper_id': '\\salmanID', 'rb_rank': '8', 'clean_acc': '52.92\\%', 'rob_acc': '25.32\\%'} I2 0.31665319683495907
{'paper_id': '\\salmanIDdue', 'rb_rank': '5', 'clean_acc': '64.02\\%', 'rob_acc': '34.96\\%'} I3 0.34617650614911694
{'paper_id': '\\wongfastID', 'rb_rank': '7', 'clean_acc': '55.62\\%', 'rob_acc': '26.24\\%'} I4 0.3952503617340699
{'paper_id': '\\engstromimagenetID', 'rb_rank': '6', 'clean_acc': '62.56\\%', 'rob_acc': '29.22\\%'} I1 0.39803588710119947


## Final ranking

In [22]:
for i in np.array(under_clean_diff).argsort():
    print(f"{model_ids[i]} -- Usece: {under_clean_diff[i]:.3f} - Osece {over_clean_diff[i]:.3f} - SECE: {clean_ece_values[i]:.3f}")

I8 -- Usece: 0.084 - Osece -0.135 - SECE: 0.157
I2 -- Usece: 0.093 - Osece -0.224 - SECE: 0.107
I7 -- Usece: 0.100 - Osece -0.156 - SECE: 0.167
I6 -- Usece: 0.106 - Osece -0.094 - SECE: 0.163
I3 -- Usece: 0.120 - Osece -0.226 - SECE: 0.076
I4 -- Usece: 0.121 - Osece -0.274 - SECE: -0.009
I5 -- Usece: 0.127 - Osece -0.175 - SECE: 0.187
I1 -- Usece: 0.137 - Osece -0.261 - SECE: 0.058


# Metrics consideration: SECE

In [23]:
clean_area = {k: v["clean_area"] for k, v in ece_dict.items()}
under_area = {k: v["under_area"] for k, v in ece_dict.items()}
over_area = {k: v["over_area"] for k, v in ece_dict.items()}

In [24]:
# Show values
for n, i in enumerate(under_area.keys()): # mettere meno, chiamiamo calibration error
    print(f"{n+1} : {-under_area[i]=:.3f}, {-clean_area[i]=:.3f}, {-over_area[i]=:.3f} --- {-(under_area[i]-clean_area[i]):.3f} // {-(over_area[i]-clean_area[i]):.3f}")

1 : -under_area[i]=0.195, -clean_area[i]=0.058, -over_area[i]=-0.203 --- 0.137 // -0.261
2 : -under_area[i]=0.200, -clean_area[i]=0.107, -over_area[i]=-0.116 --- 0.093 // -0.224
3 : -under_area[i]=0.196, -clean_area[i]=0.076, -over_area[i]=-0.150 --- 0.120 // -0.226
4 : -under_area[i]=0.112, -clean_area[i]=-0.009, -over_area[i]=-0.283 --- 0.121 // -0.274
5 : -under_area[i]=0.314, -clean_area[i]=0.187, -over_area[i]=0.012 --- 0.127 // -0.175
6 : -under_area[i]=0.270, -clean_area[i]=0.163, -over_area[i]=0.070 --- 0.106 // -0.094
7 : -under_area[i]=0.268, -clean_area[i]=0.167, -over_area[i]=0.011 --- 0.100 // -0.156
8 : -under_area[i]=0.242, -clean_area[i]=0.157, -over_area[i]=0.022 --- 0.084 // -0.135


# Metrics consideration: Entropy

In [25]:
print("Entropy values")
i=0
for e1, e2, e3 in zip(under_entropy_values, clean_entropy_values, over_entropy_values):
    print(f"{model_ids[i]} - under: {e1:.3f} // clean: {e2:.3f} // over: {e3:.3f}")
    i+=1

Entropy values
I1 - under: 3.326 // clean: 2.130 // over: 1.054
I2 - under: 3.968 // clean: 3.072 // over: 2.118
I3 - under: 3.216 // clean: 2.216 // over: 1.287
I4 - under: 3.211 // clean: 2.223 // over: 1.178
I5 - under: 3.929 // clean: 2.938 // over: 1.995
I6 - under: 3.487 // clean: 2.638 // over: 1.690
I7 - under: 3.568 // clean: 2.710 // over: 1.955
I8 - under: 3.343 // clean: 2.600 // over: 1.995


In [26]:
n_classes = 10 if ds_name == "cifar10" else 1000
max_entropy = - np.log2(1/n_classes)
print("max entropy:",max_entropy)

max entropy: 9.965784284662087


In [27]:
# MEAN ENTROPY
print("mean under entropy:",np.array(under_entropy_values).mean())
print("mean clean entropy:",np.array(clean_entropy_values).mean()) 
print("mean over entropy:",np.array(over_entropy_values).mean()) # C: 0.7185762862364451 # I: 1.659049853682518

mean under entropy: 3.5059031546115875
mean clean entropy: 2.5658993422985077
mean over entropy: 1.659049853682518


In [28]:
# mean Entropy difference
print("UNDER - CLEAN entropy:",np.array(under_entropy_values).mean() - np.array(clean_entropy_values).mean())
print("OVER - CLEAN entropy:",np.array(over_entropy_values).mean() - np.array(clean_entropy_values).mean())


UNDER - CLEAN entropy: 0.9400038123130798
OVER - CLEAN entropy: -0.9068494886159897


In [29]:
# mean Entropy difference
print("%%% UNDER - CLEAN entropy:",(np.array(under_entropy_values).mean() - np.array(clean_entropy_values).mean())/ np.array(clean_entropy_values).mean())
print("%%% OVER - CLEAN entropy:",(np.array(over_entropy_values).mean() - np.array(clean_entropy_values).mean())/ np.array(clean_entropy_values).mean())

%%% UNDER - CLEAN entropy: 0.36634477308491514
%%% OVER - CLEAN entropy: -0.3534236412421552


In [30]:
i=0
under_sum = over_sum = 0

print("Percentage displacement")

for e1, e2, e3 in zip(under_entropy_values, clean_entropy_values, over_entropy_values):
    print(f"{model_ids[i]} - under: {((e1 - e2) / e2) * 100:.3f} // over: {((e2 - e3) / e2) * 100:.3f}")
    under_sum+=((e1 - e2) / e2) * 100
    over_sum+=((e2 - e3) / e2) * 100
    i+=1
    
print(f"\navg_under: {under_sum / (i - 1):.3f} // over: {over_sum / (i - 1):.3f}")

Percentage displacement
I1 - under: 56.132 // over: 50.525
I2 - under: 29.147 // over: 31.067
I3 - under: 45.139 // over: 41.892
I4 - under: 44.464 // over: 46.994
I5 - under: 33.701 // over: 32.108
I6 - under: 32.188 // over: 35.944
I7 - under: 31.665 // over: 27.849
I8 - under: 28.570 // over: 23.264

avg_under: 43.001 // over: 41.378


In [31]:
i=0

under_diff = []
over_diff = []

for e1, e2, e3 in zip(under_entropy_values, clean_entropy_values, over_entropy_values):
    under_diff.append(e1 - e2)
    over_diff.append(e2 - e3)
    print(f"{model_ids[i]} - under: {e1 - e2:.3f} // over: {e2 - e3:.3f}")
    i+=1
    

I1 - under: 1.196 // over: 1.076
I2 - under: 0.896 // over: 0.955
I3 - under: 1.000 // over: 0.928
I4 - under: 0.988 // over: 1.045
I5 - under: 0.990 // over: 0.943
I6 - under: 0.849 // over: 0.948
I7 - under: 0.858 // over: 0.755
I8 - under: 0.743 // over: 0.605


# Weughted Uncertainty Span

In [32]:
def entropy_score(delta_entropy, clean_entropy, n_classes):
    max_entropy = - np.log2(1/n_classes)
    return np.abs(delta_entropy - clean_entropy)/(max_entropy - clean_entropy)

i=0

under_wus = []
over_wus = []

print("Entropy score ragionato con Antonio")

for e1, e2, e3 in zip(under_entropy_values, clean_entropy_values, over_entropy_values):
    under_wus.append(entropy_score(e1,e2,n_classes))
    over_wus.append(entropy_score(e3,e2,n_classes))
    print(f"{model_ids[i]} - under: {entropy_score(e1,e2,n_classes):.3f} "
          f"// over: {entropy_score(e3,e2,n_classes):.3f}")
    i+=1

Entropy score ragionato con Antonio
I1 - under: 0.153 // over: 0.137
I2 - under: 0.130 // over: 0.138
I3 - under: 0.129 // over: 0.120
I4 - under: 0.128 // over: 0.135
I5 - under: 0.141 // over: 0.134
I6 - under: 0.116 // over: 0.129
I7 - under: 0.118 // over: 0.104
I8 - under: 0.101 // over: 0.082


## WUS rank

In [33]:
uatk_arsort = np.array(under_wus).argsort()

for i, rank in enumerate(uatk_arsort):
    print(f"{model_ids[rank]} -- {under_wus[rank]:.3f}")

I8 -- 0.101
I6 -- 0.116
I7 -- 0.118
I4 -- 0.128
I3 -- 0.129
I2 -- 0.130
I5 -- 0.141
I1 -- 0.153


In [34]:
oatk_arsort = np.array(over_wus).argsort()

for i, rank in enumerate(oatk_arsort):
    print(f"{model_ids[rank]} -- {i+1} -- {over_wus[rank]:.3f}")

I8 -- 1 -- 0.082
I7 -- 2 -- 0.104
I3 -- 3 -- 0.120
I6 -- 4 -- 0.129
I5 -- 5 -- 0.134
I4 -- 6 -- 0.135
I1 -- 7 -- 0.137
I2 -- 8 -- 0.138


# WUS CONSIDERATION

In [35]:
for i, rank in enumerate(uatk_arsort):
    print(f"{model_ids[rank]} -- Uwus: {under_wus[rank]:.3f} - Owus: {over_wus[rank]:.3f} - H(x): {clean_entropy_values[rank]:.3f} - Ugap: {under_wus[rank]+over_wus[rank]:.3f}")

I8 -- Uwus: 0.101 - Owus: 0.082 - H(x): 2.600 - Ugap: 0.183
I6 -- Uwus: 0.116 - Owus: 0.129 - H(x): 2.638 - Ugap: 0.245
I7 -- Uwus: 0.118 - Owus: 0.104 - H(x): 2.710 - Ugap: 0.222
I4 -- Uwus: 0.128 - Owus: 0.135 - H(x): 2.223 - Ugap: 0.263
I3 -- Uwus: 0.129 - Owus: 0.120 - H(x): 2.216 - Ugap: 0.249
I2 -- Uwus: 0.130 - Owus: 0.138 - H(x): 3.072 - Ugap: 0.268
I5 -- Uwus: 0.141 - Owus: 0.134 - H(x): 2.938 - Ugap: 0.275
I1 -- Uwus: 0.153 - Owus: 0.137 - H(x): 2.130 - Ugap: 0.290


# Manual fix of EXCEL rank table

In [36]:
\begin{table}[]
\resizebox{\textwidth}{!}{%
\begin{tabular}{llcllclclll}
Robustbench ID & ID  & \multicolumn{1}{l}{RB ranking} & Clean acc & Rob acc & \multicolumn{1}{l}{Uatk rank} & Uatk\_value & \multicolumn{1}{l}{Oatk rank} & Oatk\_value & \multicolumn{1}{c}{} & \multicolumn{1}{c}{} \\
\cline{1-9}
Peng2023Robust & C15 & $1$ & $93.27\%$  & $71.07\%$ & $3$ & $0.100$ & $2$ & $-0.071$ \\
Wang2023Better_WRN-70-16 & C14 & $2$ & $93.25\%$  & $70.69\%$ & $4$ & $0.103$ & $3$ & $-0.073$ \\
Cui2023Decoupled_WRN-28-10 & C6 & $3$ & $92.16\%$ & $67.73\%$ & $2$ & $0.099$ & $4$ & $-0.077$ \\
Wang2023Better_WRN_28_10 & C8 & $4$ & $92.44\%$ & $67.31\%$ & $5$ & $0.106$ & $5$ & $-0.079$ \\
Rebuffi2021Fixing_70_16_cutmix_extra & C13 & $5$ & $92.23\%$  & $66.56\%$ & $12$ & $0.137$ & $9$ & $-0.101$ \\
Gowal2021Improving_70_16_ddpm_100m & C10 & $6$ & $88.74\%$  & $66.10\%$ & $8$ & $0.112$ & $7$ & $-0.089$ \\
Kang2021Stable & C11 & $7$ & $93.73\%$ & $64.20\%$ & $1$ & $0.017$ & $1$ & $-0.031$ \\
Xu2023Exploring_WRN_28_10 & C9 & $8$ & $93.69\%$ & $63.89\%$ & $15$ & $0.221$ & $15$ & $-0.157$ \\
Gowal2021Improving_28_10_ddpm_100m & C7 & $9$ & $87.50\%$ & $63.38\%$ & $10$ & $0.126$ & $10$ & $-0.102$ \\
Pang2022Robustness_WRN70_16 & C12 & $10$ & $89.01\%$ & $63.35\%$ & $7$ & $0.111$ & $6$ & $-0.087$ \\
sehwag2021Proxy_ResNest152 & C5 & $11$ & $87.30\%$ & $62.79\%$ & $9$ & $0.120$ & $11$ & $-0.102$ \\
Sehwag2021Proxy_R18 & C3 & $12$ & $84.59\%$ & $55.54\%$ & $13$ & $0.142$ & $12$ & $-0.123$ \\
Addepalli2022Efficient_RN18 & C1 & $13$ & $85.71\%$ & $52.48\%$ & $6$ & $0.106$ & $8$ & $-0.091$ \\
Addepalli2021Towards_RN18 & C2 & $14$ & $80.24\%$ & $51.06\%$ & $11$ & $0.132$ & $13$ & $-0.130$ \\
Engstrom2019Robustness & C4 & $15$ & $87.03\%$  & $49.25\%$ & $14$ & $0.170$ & $14$ & $-0.134$ \\
\end{tabular}}
\caption{\cifar sum up with attack delta and ranking by minimum delta}
\end{table}

\begin{table}[]
\resizebox{\textwidth}{!}{\%
\begin{tabular}{lllllllllll}
Robustbench ID & ID & RB ranking  & Clean acc & Rob acc & Uatk rank   & Uatk\_value & Oatk rank   & Oatk\_value & \multicolumn{1}{c}{} & \multicolumn{1}{c}{} \\ \cline{1-9}
Liu2023SwinL & I6 & $1$ & $78.92\%$  & $59.56\%$ & $2$ & $0.116$ & $4$ & $-0.129$ \\
Liu2023ConvNextL & I8 & $2$ & $78.02\%$  & $58.48\%$ & $1$ & $0.101$ & $1$ & $-0.082$ \\
Liu2023swinB & I5 & $3$ & $76.16\%$  & $56.16\%$ & $7$ & $0.141$ & $5$ & $-0.134$ \\
Liu2023ConvNextB & I7 & $4$ & $76.02\%$  & $55.82\%$ & $3$ & $0.118$ & $2$ & $-0.104$ \\
Salman2020R50 & I3 & $5$ & $64.02\%$  & $34.96\%$ & $5$ & $0.129$ & $3$ & $-0.12$ \\
Engstrom2019imgnet & I2 & $6$ & $62.56\%$  & $29.22\%$ & $8$ & $0.153$ & $7$ & $-0.137$ \\
Wong2020 & I4 & $7$ & $55.62\%$  & $26.24\%$ & $4$ & $0.128$ & $6$ & $-0.135$ \\
Salman2020R18 & I1 & $8$ & $52.92\%$  & $25.32\%$ & $6$ & $0.13$ & $8$ & $-0.138$ \\
\end{tabular}}
\caption{\imagenet sum up with attack delta and ranking by minimum delta}
\end{table}


SyntaxError: unexpected character after line continuation character (2023530864.py, line 1)